In [4]:
!pip install git+https://github.com/AI4Finance-LLC/FinRL-Library.git

  Cloning https://github.com/AI4Finance-LLC/FinRL-Library.git to c:\users\user\appdata\local\temp\pip-req-build-eljb4yz0
  Cloning https://github.com/quantopian/pyfolio.git to c:\users\user\appdata\local\temp\pip-install-fxee6hdm\pyfolio_41b3a03783c24732aa16556b32d3fcda

  Running command git clone -q https://github.com/AI4Finance-LLC/FinRL-Library.git 'C:\Users\User\AppData\Local\Temp\pip-req-build-eljb4yz0'
  Running command git clone -q https://github.com/quantopian/pyfolio.git 'C:\Users\User\AppData\Local\Temp\pip-install-fxee6hdm\pyfolio_41b3a03783c24732aa16556b32d3fcda'



  Created wheel for finrl: filename=finrl-0.3.0-py3-none-any.whl size=38613 sha256=c24392ec67de8187a11b56caa1c7a12eb470a6c49ac756b36b8eea2ec27458e2
  Stored in directory: C:\Users\User\AppData\Local\Temp\pip-ephem-wheel-cache-me1p7zvk\wheels\e8\19\74\11261997d6bdca44ba73e8eeedb94a3e3d340259516a0887eb
  Created wheel for pyfolio: filename=pyfolio-0.9.2+75.g4b901f6-py3-none-any.whl size=76260 sha256=653d9635bfbd552d2017ddf62a73f2514e4119e9096ce2967e34717186a1d50c
  Stored in directory: C:\Users\User\AppData\Local\Temp\pip-ephem-wheel-cache-me1p7zvk\wheels\7b\59\8b\3c276a18b58c04a1fd0e1351e979fb5396f93fbde5b5438df1
Successfully built finrl pyfolio
  Attempting uninstall: pyfolio
    Found existing installation: pyfolio 0.9.2
    Uninstalling pyfolio-0.9.2:
      Successfully uninstalled pyfolio-0.9.2


In [ ]:
pip install stable_baselines

In [9]:
# import packages
import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
matplotlib.use('Agg')
import datetime

from finrl.config import config
from finrl.marketdata.yahoodownloader import YahooDownloader
from finrl.preprocessing.preprocessors import FeatureEngineer
from finrl.preprocessing.data import data_split
from finrl.env.environment import EnvSetup
from finrl.env.EnvMultipleStock_train import StockEnvTrain
from finrl.env.EnvMultipleStock_trade import StockEnvTrade
from finrl.model.models import DRLAgent
from finrl.trade.backtest import backtest_stats, backtest_plot, get_daily_return, get_baseline

import os
if not os.path.exists("./" + config.DATA_SAVE_DIR):
    os.makedirs("./" + config.DATA_SAVE_DIR)
if not os.path.exists("./" + config.TRAINED_MODEL_DIR):
    os.makedirs("./" + config.TRAINED_MODEL_DIR)
if not os.path.exists("./" + config.TENSORBOARD_LOG_DIR):
    os.makedirs("./" + config.TENSORBOARD_LOG_DIR)
if not os.path.exists("./" + config.RESULTS_DIR):
    os.makedirs("./" + config.RESULTS_DIR)

In [10]:
# Download and save the data in a pandas DataFrame:
df = YahooDownloader(start_date = '2010-01-01',
                     end_date = '2018-12-01',
                     ticker_list = config.JII_TICKER).fetch_data()


[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%********

In [13]:
# Perform Feature Engineering:
df = FeatureEngineer(df.copy()).preprocess_data()


# add covariance matrix as states
df=df.sort_values(['date','tic'],ignore_index=True)
df.index = df.date.factorize()[0]

cov_list = []
# look back is one year
lookback=252
for i in range(lookback,len(df.index.unique())):
  data_lookback = df.loc[i-lookback:i,:]
  price_lookback=data_lookback.pivot_table(index = 'date',columns = 'tic', values = 'close')
  return_lookback = price_lookback.pct_change().dropna()
  covs = return_lookback.cov().values 
  cov_list.append(covs)
  
df_cov = pd.DataFrame({'date':df.date.unique()[lookback:],'cov_list':cov_list})
df = df.merge(df_cov, on='date')
df = df.sort_values(['date','tic']).reset_index(drop=True)
df.head() 

TypeError: preprocess_data() missing 1 required positional argument: 'df'

In [ ]:
env_setup = EnvSetup(stock_dim = stock_dimension,
                        state_space = state_space,
                        initial_amount = 1000000,
                        tech_indicator_list = config.TECHNICAL_INDICATORS_LIST)
                        
env_train = env_setup.create_env_training(data = train,
                                          env_class = StockPortfolioEnv)                        
